1584. Min Cost to Connect All Points

Medium

You are given an array points representing integer coordinates of some points on a 2D-plane, where points[i] = [xi, yi].

The cost of connecting two points [xi, yi] and [xj, yj] is the manhattan distance between them: |xi - xj| + |yi - yj|, where |val| denotes the absolute value of val.

Return the minimum cost to make all points connected. All points are connected if there is exactly one simple path between any two points.
---
I was able to understand some of the problem after a lot of fiddling with new concepts. However, the way I solved is not entirely correct and I am unsure how to properly fix it at it's current state. I think it is time to learn a new concept in the form of heaps. Which I've been putting off for so long now!



In [ ]:
# orginial correct code, that is far too slow.
class Solution:
    def minCostConnectPoints(self, points: List[List[int]]) -> int:
        # first build all edges with weights
        if len(points) == 1:
            return 0
        adj = collections.defaultdict(list)
        minDist = float("inf")
        minPair = None
        for i in range(len(points)):
            for j in range(i+1,len(points)):
                x,y = points[i]
                xx,yy = points[j]
                manDist = abs(x-xx) + abs(y-yy)
                if manDist < minDist:
                    minDist = manDist
                    minPair = (i,j)
                adj[i].append((manDist,j))
                adj[j].append((manDist, i))
        edges = set()
        edges.add(minPair)
        edges.add((minPair[1],minPair[0]))
        print(edges)
        weight = minDist  
        pointSet = set([minPair[0],minPair[1]])
        while len(edges) // 2 < (len(points)-1):
            newMin = float("inf")
            newPoint = None
            newEdge = None
            for point in pointSet:
                for dist, index in adj[point]:
                    if (point,index) not in edges and index not in pointSet:
                        if dist < newMin:
                            newMin = dist
                            newPoint = index
                            newEdge = (point,index)
            pointSet.add(newPoint)
            weight += newMin
            edges.add(newEdge)
            edges.add((newEdge[1],newEdge[0]))
        return weight


I was able to correctly implement the code, but there is a major bottle neck for performance. That bottle neck is how ever iteration I am re-checking every single edge that exists. I instead need to do two things:
1. implement a heap
2. when I add a new point to the graph, I need to push it's edges onto the heap
But using heaps is an idea that is foreign to me, and I must learn!


In [ ]:
# orginial correct code, that is far too slow.
import heapq
class Solution:
    def minCostConnectPoints(self, points: List[List[int]]) -> int:
        # first build all edges with weights
        adj = collections.defaultdict(list)
        minDist = float("inf")
        minPair = None
        for i in range(len(points)):
            for j in range(i+1,len(points)):
                x,y = points[i]
                xx,yy = points[j]
                manDist = abs(x-xx) + abs(y-yy)
                if manDist < minDist:
                    minDist = manDist
                    minPair = (i,j)
                adj[i].append((manDist,j))
                adj[j].append((manDist, i))
        edges = set()
        edges.add(minPair)
        edges.add((minPair[1],minPair[0]))
        weight = minDist  
        pointSet = set([minPair[0],minPair[1]])
        heap = []
        for point in pointSet:
            for dist, index in adj[point]:
                if (point,index) not in edges and index not in pointSet:
                    heapq.heappush(heap,(dist,(point,index)))
        print(heap)

        while len(edges) // 2 < (len(points)-1):
            smallest, edge = heapq.heappop(heap)
            if edge[1] in pointSet:
                continue
            newMin = smallest
            newPoint = edge[1]
            newEdge = edge
            for dist, index in adj[point]:
                if (newPoint,index) not in edges and index not in pointSet:
                    heapq.heappush(heap,(dist,(point,index)))
            pointSet.add(newPoint)
            weight += newMin
            edges.add(newEdge)
            edges.add((newEdge[1],newEdge[0]))
        return weight


In [ ]:
# last attempt at a severely optimized version of my code
import heapq

class Solution:
    def minCostConnectPoints(self, points: List[List[int]]) -> int:
        n = len(points)
        if n == 1:
            return 0
        
        # Calculate the Manhattan distance between two points
        def manhattan_distance(p1, p2):
            return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])
        
        # initialize variables
        minCost = 0
        start = points[0]
        visited = set()
        heap = []

        for i in range(1,n):
            heapq.heappush(heap,(manhattan_distance(start,points[i]),i))
        
        visited.add(0)

        while len(visited) < n:
            smallest, index = heapq.heappop(heap)
            
            if index in visited:
                continue
            
            visited.add(index)

            minCost += smallest

            for i in range(n):
                if i not in visited:
                    heapq.heappush(heap,(manhattan_distance(points[index],points[i]),i))
        return minCost


Okay, this code largely makes sense to me now after working for so long on it. I am happy that I took the time to learn these important skills!
Overall, the dominant factor in the time complexity is the main loop, which has a time complexity of O(n*log(n)).
Overall, the space complexity of the code is O(n) due to the heap and the visited set, where n is the number of points in the input list.